# Práctica 1 opcional

In [ ]:
import cv2 as cv
import numpy as np

# --- Cargar imagen ---
file = 'images/eii.png'
image = cv.imread(file)
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")

canvas = image.copy()
drawing = False
ix, iy = -1, -1
points = []  # para polilíneas y polígonos

# --- Ventana principal ---
cv.namedWindow("Editor", cv.WINDOW_AUTOSIZE)

# --- Trackbars ---
def nothing(x): pass

cv.createTrackbar("Figura", "Editor", 0, 4, nothing)  # 0=linea,1=rect,2=circle,3=polyline,4=polygon
cv.createTrackbar("B", "Editor", 0, 255, nothing)
cv.createTrackbar("G", "Editor", 0, 255, nothing)
cv.createTrackbar("R", "Editor", 255, 255, nothing)
cv.createTrackbar("Grosor", "Editor", 2, 20, nothing)
cv.createTrackbar("Relleno", "Editor", 0, 1, nothing)

def draw(event, x, y, flags, param):
    global ix, iy, drawing, canvas, image, points

    figura = cv.getTrackbarPos("Figura", "Editor")
    b = cv.getTrackbarPos("B", "Editor")
    g = cv.getTrackbarPos("G", "Editor")
    r = cv.getTrackbarPos("R", "Editor")
    color = (b, g, r)

    thickness = cv.getTrackbarPos("Grosor", "Editor")
    relleno = cv.getTrackbarPos("Relleno", "Editor")

    # ⚠️ grosor -1 solo en figuras cerradas
    if figura not in [0, 3] and relleno == 1:
        thickness = -1

    # --------- Dibujos normales ----------
    if figura in [0, 1, 2]:
        if event == cv.EVENT_LBUTTONDOWN:
            drawing = True
            ix, iy = x, y

        elif event == cv.EVENT_MOUSEMOVE and drawing:
            temp = image.copy()
            if figura == 0:   # línea
                cv.line(temp, (ix, iy), (x, y), color, thickness)
            elif figura == 1: # rectángulo
                cv.rectangle(temp, (ix, iy), (x, y), color, thickness)
            elif figura == 2: # círculo
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv.circle(temp, (ix, iy), radius, color, thickness)
            canvas = temp

        elif event == cv.EVENT_LBUTTONUP:
            drawing = False
            if figura == 0:
                cv.line(image, (ix, iy), (x, y), color, thickness)
            elif figura == 1:
                cv.rectangle(image, (ix, iy), (x, y), color, thickness)
            elif figura == 2:
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv.circle(image, (ix, iy), radius, color, thickness)
            canvas = image.copy()

    # --------- Polilínea / Polígono ----------
    elif figura in [3, 4]:
        if event == cv.EVENT_LBUTTONDOWN:
            points.append((x, y))

        elif event == cv.EVENT_MOUSEMOVE and len(points) > 0:
            temp = image.copy()
            # usamos grosor positivo para vista previa
            preview_thickness = thickness if thickness > 0 else 2

            # dibujar la polilínea fija con puntos confirmados
            cv.polylines(temp, [np.array(points)], False, color, preview_thickness)
            # goma elástica hasta el cursor
            cv.line(temp, points[-1], (x, y), color, preview_thickness)
            # dibujar puntos de control
            for p in points:
                cv.circle(temp, p, 4, (0, 0, 0), -1)
                cv.circle(temp, p, 2, (255, 255, 255), -1)
            canvas = temp


        elif event == cv.EVENT_RBUTTONDOWN and len(points) > 1:
            if figura == 3:  # polilínea abierta
                cv.polylines(image, [np.array(points)], False, color, max(1, thickness))
            elif figura == 4:  # polígono cerrado
                if thickness == -1:  # relleno
                    cv.fillPoly(image, [np.array(points)], color)
                else:
                    cv.polylines(image, [np.array(points)], True, color, thickness)
            canvas = image.copy()
            points = []  # reiniciar


# Asociar callback
cv.setMouseCallback("Editor", draw)

print("\n👉 Usa sliders para cambiar figura, color, grosor y relleno.")
print("0=Línea, 1=Rectángulo, 2=Círculo, 3=Polilínea, 4=Polígono")
print("En polilínea/polígono: clics izquierdos = puntos, clic derecho = cerrar figura.")
print("ESC para salir.")

height, width = canvas.shape[:2]
fourcc = cv.VideoWriter_fourcc(*'XVID')  # codec AVI
out = cv.VideoWriter('output/dibujo.avi', fourcc, 20.0, (width, height))

# Loop principal
while True:
    cv.imshow("Editor", canvas)
    out.write(canvas)  # <-- guardar frame
    if cv.waitKey(1) & 0xFF == 27:  # ESC
        break

out.release()
cv.destroyAllWindows()



👉 Usa sliders para cambiar figura, color, grosor y relleno.
0=Línea, 1=Rectángulo, 2=Círculo, 3=Polilínea, 4=Polígono
En polilínea/polígono: clics izquierdos = puntos, clic derecho = cerrar figura.
ESC para salir.
